# SpectraMind V50 — Mini Pipeline

Calibrate → Train (1 epoch) → Predict (submission CSV).

**Note**: For the full run, use `make e2e`. This notebook runs a tiny version suitable for local demo.

In [ ]:
import os, shlex, subprocess, sys
from pathlib import Path

def choose_cli() -> str:
    def ok(cmd: str) -> bool:
        try:
            subprocess.run(cmd + " --help", shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            return True
        except Exception:
            return False
    for cmd in ("poetry run spectramind", "spectramind", f"{shlex.quote(sys.executable)} -m spectramind"):
        if ok(cmd):
            return cmd
    return "spectramind"

CLI = choose_cli()
OUT_DIR = Path("outputs")
DIAG_DIR = OUT_DIR / "diagnostics"
OUT_DIR.mkdir(exist_ok=True)
DIAG_DIR.mkdir(parents=True, exist_ok=True)
CLI

## 1) Calibrate (toy / fast)
Produces science‑ready light‑curves/spectra in `outputs/calibrated/`.

In [ ]:
subprocess.run(CLI + " calibrate +data.split=toy", shell=True, check=False)

## 2) Train (1 epoch, light mode)
Writes checkpoints and logs to `outputs/checkpoints/`.

In [ ]:
subprocess.run(CLI + " train +training.epochs=1 +diagnostics.light=true", shell=True, check=False)

## 3) Predict
Generates a submission CSV: `outputs/submission.csv`.

In [ ]:
subprocess.run(CLI + " predict --out-csv outputs/submission.csv", shell=True, check=False)
Path("outputs/submission.csv").exists()